In [49]:
import pandas as pd
import numpy as np
import re
import json

from jupyter_server.services.config.handlers import section_name_regex
from scipy.signal import vectorstrength
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [50]:
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)

try:
    nlp = spacy.load('en_core_web_md')
except:
    nlp = spacy.load('en_core_web_sm')

In [51]:
import pandas as pd
import numpy as np
import re
import json
import time
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class InsuranceTaxonomyClassifier:
    """
    Clasificator care potrivește companii cu etichete din taxonomia de asigurări.
    Utilizez tehnici NLP simple și potrivire bazată pe similaritate.
    """

    def __init__(self, taxonomy_path, threshold=0.15):
        """
        Inițializez clasificatorul cu datele taxonomiei și pragul de potrivire.

        Args:
            taxonomy_path (str): Calea către fișierul CSV cu taxonomia
            threshold (float): Pragul de similaritate pentru potrivire
        """
        print(f"[{self._get_timestamp()}] Inițializare clasificator...")

        self.threshold = threshold

        # Încarc datele taxonomiei
        self.taxonomy_df = pd.read_csv(taxonomy_path)
        self.taxonomy_labels = self.taxonomy_df['label'].tolist()

        # Creez mapări pentru domenii specifice
        self.domain_mappings = self._create_domain_mappings()

        # Procesez etichetele taxonomiei
        print(f"[{self._get_timestamp()}] Procesare {len(self.taxonomy_labels)} etichete din taxonomie...")
        self.processed_labels = self._preprocess_taxonomy()

        # Inițializez vectorizatorul TF-IDF
        self.vectorizer = TfidfVectorizer(
            analyzer='word',
            min_df=2,
            max_df=0.95,
            ngram_range=(1, 2)  # Include unigrame și bigrame
        )

        # Pregătesc vectorizatorul cu textul din taxonomie
        all_taxonomy_text = [' '.join(label_info['keywords']) for label_info in self.processed_labels]
        self.vectorizer.fit(all_taxonomy_text)

        # Transform etichetele taxonomiei în vectori TF-IDF
        self.taxonomy_vectors = self.vectorizer.transform(all_taxonomy_text)

        print(f"[{self._get_timestamp()}] Clasificator inițializat cu succes!")

    def _get_timestamp(self):
        """Returnează timestamp-ul curent formatat."""
        return datetime.now().strftime("%H:%M:%S")

    def _create_domain_mappings(self):
        """Creez mapări de cuvinte cheie specifice domeniului pentru o mai bună înțelegere a contextului."""
        return {
            'construction': ['building', 'contractor', 'engineering', 'infrastructure', 'development', 'project'],
            'healthcare': ['medical', 'health', 'clinic', 'hospital', 'patient', 'care', 'wellness', 'therapy'],
            'agriculture': ['farm', 'crop', 'plant', 'soil', 'harvest', 'cultivation', 'field', 'livestock'],
            'technology': ['software', 'hardware', 'it', 'computer', 'digital', 'tech', 'application', 'system'],
            'manufacturing': ['production', 'factory', 'fabrication', 'assembly', 'industrial', 'processing'],
            'retail': ['store', 'shop', 'sales', 'customer', 'merchandise', 'consumer', 'mall'],
            'food': ['restaurant', 'meal', 'cuisine', 'culinary', 'catering', 'dietary', 'nutrition'],
            'transportation': ['logistics', 'shipping', 'freight', 'delivery', 'hauling', 'trucking', 'transport'],
            'education': ['school', 'training', 'learning', 'teaching', 'academic', 'educational', 'instruction'],
            'finance': ['banking', 'investment', 'financial', 'money', 'fiscal', 'economic', 'accounting'],
            'legal': ['law', 'attorney', 'legal', 'counsel', 'litigation', 'judicial', 'regulatory'],
            'energy': ['power', 'electricity', 'gas', 'utilities', 'fuel', 'renewable', 'energy'],
            'real_estate': ['property', 'real estate', 'leasing', 'rental', 'tenant', 'building', 'housing'],
            'automotive': ['vehicle', 'car', 'auto', 'automotive', 'repair', 'mechanic', 'motor'],
            'design': ['designer', 'creative', 'aesthetic', 'artistic', 'layout', 'visual'],
            'consulting': ['advisory', 'consultant', 'guidance', 'counsel', 'strategy', 'recommendation'],
            'media': ['publishing', 'broadcast', 'film', 'production', 'content', 'entertainment'],
            'cosmetics': ['beauty', 'makeup', 'skincare', 'salon', 'spa', 'cosmetic'],
            'sports': ['fitness', 'athletic', 'exercise', 'gym', 'sport', 'recreation'],
            'maintenance': ['repair', 'upkeep', 'service', 'maintain', 'fix', 'cleaning'],
            'animal': ['pet', 'veterinary', 'animal', 'livestock', 'fauna', 'wildlife'],
            'security': ['protection', 'guard', 'surveillance', 'safety', 'alarm', 'monitor']
        }

    def _preprocess_text(self, text):
        """Procesez textul prin eliminarea caracterelor speciale, normalizare etc."""
        if not isinstance(text, str):
            return ""

        # Convertesc la litere mici și elimină caractere speciale
        text = text.lower()
        text = re.sub(r'[^\w\s]', ' ', text)

        # Elimin cuvinte comune foarte scurte
        words = [word for word in text.split() if len(word) > 2]

        return ' '.join(words)

    def _parse_business_tags(self, tags_str):
        """Analizez tagurile de business din reprezentarea string."""
        try:
            # Înlocuiesc ghilimele simple cu duble pentru parsarea JSON
            fixed_json = tags_str.replace("'", '"')
            return json.loads(fixed_json)
        except:
            if isinstance(tags_str, str) and tags_str.startswith('[') and tags_str.endswith(']'):
                # Extrag elementele dintre paranteze și împart după virgulă
                items = tags_str[1:-1].split(',')
                # Curăț fiecare element
                return [item.strip().strip("'").strip('"') for item in items]
            return []

    def _preprocess_taxonomy(self):
        """Procesez etichetele taxonomiei pentru a extrage caracteristici și domenii cheie."""
        processed_labels = []

        for label in self.taxonomy_labels:
            # Procesare de bază a textului
            clean_label = self._preprocess_text(label)

            # Creez variație fără sufixul "Services"
            without_services = re.sub(r'\sservices$', '', clean_label)

            # Extrag cuvinte
            words = [word for word in clean_label.split() if len(word) > 2]

            # Identific domeniile relevante
            related_domains = []
            domain_keywords = []

            # Identific domeniile relevante
            for domain, keywords in self.domain_mappings.items():
                if any(keyword in clean_label for keyword in keywords):
                    related_domains.append(domain)
                    domain_keywords.extend(keywords)

            # Creeaz lista de cuvinte cheie
            keywords = words + [word for word in domain_keywords if word not in words]

            # Creeaz bigrame pentru un context mai bun
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(f"{words[i]} {words[i+1]}")

            processed_labels.append({
                'original': label,
                'clean_label': clean_label,
                'without_services': without_services,
                'words': words,
                'bigrams': bigrams,
                'keywords': keywords + bigrams,  # Combină pentru o reprezentare mai bogată
                'related_domains': related_domains
            })

        return processed_labels

    def _preprocess_company(self, company):
        """Procesez datele companiei pentru clasificare."""
        # Analizez tagurile de business
        if isinstance(company['business_tags'], str):
            business_tags = self._parse_business_tags(company['business_tags'])
        else:
            business_tags = []

        # Curăț câmpurile de text
        clean_description = self._preprocess_text(company['description'])
        clean_tags = self._preprocess_text(' '.join(business_tags))
        clean_sector = self._preprocess_text(company['sector'])
        clean_category = self._preprocess_text(company['category'])
        clean_niche = self._preprocess_text(company['niche'])

        # Creez text combinat pentru analiză (cu ponderare câmpuri)
        combined_text = (
            f"{clean_description} {clean_description} "  # Pondere dublă pentru descriere
            f"{clean_tags} {clean_tags} {clean_tags} "  # Pondere triplă pentru taguri de business
            f"{clean_sector} {clean_sector} "           # Pondere dublă pentru sector
            f"{clean_category} {clean_category} "       # Pondere dublă pentru categorie
            f"{clean_niche} {clean_niche}"              # Pondere dublă pentru nișă
        )

        # Extrage cuvinte cheie specifice domeniului
        domain_presence = {}
        for domain, keywords in self.domain_mappings.items():
            matches = sum(keyword in combined_text for keyword in keywords)
            if matches > 0:
                domain_presence[domain] = matches

        # Creez vector de document folosind TF-IDF
        company_vector = self.vectorizer.transform([combined_text])

        return {
            'original': company,
            'business_tags': business_tags,
            'clean_description': clean_description,
            'clean_tags': clean_tags,
            'clean_sector': clean_sector,
            'clean_category': clean_category,
            'clean_niche': clean_niche,
            'combined_text': combined_text,
            'domain_presence': domain_presence,
            'vector': company_vector
        }

    def _score_company_for_label(self, processed_company, label_info):
        """
        Calculez un scor complet de similaritate între o companie și o etichetă din taxonomie.
        Utilizez similaritate vectorială, potriviri exacte și înțelegere de domeniu.
        """
        score = 0

        # Similaritate vectorială (similaritate cosinus TF-IDF)
        vector_similarity = cosine_similarity(
            processed_company['vector'],
            self.vectorizer.transform([' '.join(label_info['keywords'])])
        )[0][0]

        # Scalează până la procent și adaugă la scor
        score += vector_similarity * 50

        # Verific potriviri exacte
        clean_label = label_info['clean_label']
        without_services = label_info['without_services']

        # Verific potriviri exacte
        if clean_label in processed_company['combined_text']:
            score += 25
        elif without_services != clean_label and without_services in processed_company['combined_text']:
            score += 20

        # Potrivirea tagurilor de business (pondere mai mare pentru potriviri exacte în taguri)
        if any(clean_label in tag.lower() for tag in processed_company['business_tags']):
            score += 20
        elif any(without_services in tag.lower() for tag in processed_company['business_tags']):
            score += 15

        # Potrivire la nivel de cuvânt
        for word in label_info['words']:
            if word in processed_company['combined_text']:
                score += 2

            # Pondere extra pentru tagurile de business care conțin termeni cheie
            if any(word in tag.lower() for tag in processed_company['business_tags']):
                score += 3

        # Potrivire bigrame (mai bună pentru context)
        for bigram in label_info['bigrams']:
            if bigram in processed_company['combined_text']:
                score += 5

        # Scorare de relevanță pentru domeniu
        for domain in label_info['related_domains']:
            if domain in processed_company['domain_presence']:
                # Potrivire de domeniu - mai puternică dacă ambele au prezență ridicată
                score += 2 * min(3, processed_company['domain_presence'][domain])

        # Scoruri pentru potrivirea câmpurilor cheie (sector, categorie, nișă)
        fields = {'sector': processed_company['clean_sector'],
                 'category': processed_company['clean_category'],
                 'niche': processed_company['clean_niche']}

        for field_name, field_value in fields.items():
            if clean_label in field_value or without_services in field_value:
                # Pondere mai mare pentru potrivirile din câmpuri structurale
                score += 8

            # Verific și potriviri la nivel de cuvânt în aceste câmpuri
            for word in label_info['words']:
                if word in field_value:
                    score += 3

        return score

    def classify_company(self, company, top_n=3):
        """
        Clasific o companie în cele mai relevante etichete din taxonomia de asigurări.

        Args:
            company (dict): Datele companiei cu descriere, business_tags, sector, categorie, nișă
            top_n (int): Numărul de potriviri de top de returnat

        Returns:
            dict: Rezultatele clasificării cu etichete potrivite și scoruri
        """
        # Preprocesez datele companiei
        processed_company = self._preprocess_company(company)

        # Calculez scoruri pentru fiecare etichetă din taxonomie
        scores = []
        for idx, label_info in enumerate(self.processed_labels):
            score = self._score_company_for_label(processed_company, label_info)
            scores.append({
                'label': label_info['original'],
                'score': score
            })

        # Sortez după scor în ordine descrescătoare
        sorted_scores = sorted(scores, key=lambda x: x['score'], reverse=True)

        # Obțin primele N potriviri
        top_matches = sorted_scores[:top_n]

        # Filtrez potrivirile peste prag
        matches_above_threshold = [
            match for match in sorted_scores
            if match['score'] >= self.threshold * 100  # Convertește pragul la aceeași scară ca scorurile
        ]

        # Mă asigur că returnez cel puțin o etichetă (cea mai bună potrivire)
        if not matches_above_threshold:
            matches_above_threshold = [sorted_scores[0]]

        return {
            'top_matches': top_matches,
            'matches_above_threshold': matches_above_threshold
        }

    def classify_companies(self, companies_df):
        """
        Clasific mai multe companii într-un DataFrame.

        Args:
            companies_df (DataFrame): DataFrame care conține datele companiilor

        Returns:
            DataFrame: DataFrame-ul original cu coloana insurance_label adăugată
        """
        # Creez o copie pentru a evita modificarea originalului
        result_df = companies_df.copy()

        # Adaug coloane pentru etichete de asigurare și scoruri de încredere
        result_df['insurance_label'] = None
        result_df['confidence_score'] = None

        # Timp total de procesare și statistici
        start_time = time.time()
        total_companies = len(result_df)

        print(f"[{self._get_timestamp()}] Începerea clasificării pentru {total_companies} companii...")

        # Procesez fiecare companie
        for idx, row in result_df.iterrows():
            company_dict = row.to_dict()
            classification = self.classify_company(company_dict)

            # Obțin etichetele potrivite și scorurile
            matched_items = classification['matches_above_threshold']
            matched_labels = [match['label'] for match in matched_items]

            # Unesc etichetele multiple cu punct și virgulă
            result_df.at[idx, 'insurance_label'] = '; '.join(matched_labels)

            # Adaug scorul de încredere (media scorurilor de top)
            if matched_items:
                avg_confidence = sum(match['score'] for match in matched_items) / len(matched_items)
                result_df.at[idx, 'confidence_score'] = round(avg_confidence, 2)
            else:
                result_df.at[idx, 'confidence_score'] = 0

            # Afișez progresul pentru fiecare 100 de companii
            if idx % 100 == 0 or idx == total_companies - 1:
                elapsed = time.time() - start_time
                rate = (idx + 1) / elapsed if elapsed > 0 else 0
                remaining = (total_companies - idx - 1) / rate if rate > 0 else 0

                print(f"[{self._get_timestamp()}] Procesate {idx+1}/{total_companies} companii " +
                     f"({rate:.1f} companii/sec, est. {remaining/60:.1f} min rămase)")

        # Statistici finale
        elapsed_time = time.time() - start_time
        avg_rate = total_companies / elapsed_time

        print(f"[{self._get_timestamp()}] Clasificare finalizată în {elapsed_time:.1f} secunde!")
        print(f"[{self._get_timestamp()}] Viteză medie de procesare: {avg_rate:.1f} companii/secundă")

        # Calculez statistici de etichete
        unique_labels = set()
        for labels in result_df['insurance_label'].str.split('; '):
            unique_labels.update(labels)

        multi_label_count = sum(result_df['insurance_label'].str.contains(';'))

        print(f"[{self._get_timestamp()}] Statistici finale:")
        print(f"  - Etichete unice utilizate: {len(unique_labels)}/{len(self.taxonomy_labels)} ({len(unique_labels)/len(self.taxonomy_labels)*100:.1f}%)")
        print(f"  - Companii cu etichete multiple: {multi_label_count} ({multi_label_count/total_companies*100:.1f}%)")

        return result_df

    def evaluate_performance(self, test_companies, sample_size=200):
        """
        Evaluez performanța clasificatorului pe un eșantion de companii.

        Args:
            test_companies (DataFrame): Companiile pentru evaluare
            sample_size (int): Numărul de companii de eșantionat

        Returns:
            dict: Metrici de performanță
        """
        print(f"[{self._get_timestamp()}] Evaluarea performanței pe un eșantion de {sample_size} companii...")

        if len(test_companies) > sample_size:
            sample = test_companies.sample(sample_size, random_state=42)
        else:
            sample = test_companies

        # Procesez eșantionul
        results = []
        start_time = time.time()

        for idx, row in sample.iterrows():
            company_dict = row.to_dict()
            classification = self.classify_company(company_dict)

            top_match = classification['top_matches'][0]
            matches_count = len(classification['matches_above_threshold'])

            results.append({
                'company_idx': idx,
                'sector': company_dict['sector'],
                'category': company_dict['category'],
                'top_match': top_match['label'],
                'top_score': top_match['score'],
                'matches_count': matches_count,
                'all_matches': [match['label'] for match in classification['matches_above_threshold']]
            })

        # Calculez metricile de performanță
        results_df = pd.DataFrame(results)

        # Distribuția etichetelor
        label_counts = results_df['top_match'].value_counts().to_dict()

        # Numărul mediu de potriviri per companie
        avg_matches = results_df['matches_count'].mean()

        # Distribuția pe sectoare
        sector_distribution = results_df.groupby('sector')['top_match'].apply(list).to_dict()

        # Analizez diversitatea potrivirilor
        all_match_labels = [label for matches in results_df['all_matches'] for label in matches]
        unique_labels_assigned = len(set(all_match_labels))
        label_coverage = unique_labels_assigned / len(self.taxonomy_labels) * 100

        elapsed_time = time.time() - start_time

        print(f"[{self._get_timestamp()}] Evaluare finalizată în {elapsed_time:.2f} secunde")
        print(f"[{self._get_timestamp()}] Rezultate evaluare:")
        print(f"  - Număr mediu de potriviri per companie: {avg_matches:.2f}")
        print(f"  - Etichete unice atribuite: {unique_labels_assigned} ({label_coverage:.1f}% din taxonomie)")
        print(f"  - Top 5 etichete atribuite:")

        for label, count in sorted(label_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"    - {label}: {count}")

        return {
            'label_distribution': label_counts,
            'avg_matches_per_company': avg_matches,
            'sector_distribution': sector_distribution,
            'unique_labels_assigned': unique_labels_assigned,
            'label_coverage_percentage': label_coverage,
            'sample_results': results
        }


def main():
    """Funcția principală pentru rularea clasificatorului."""

    # Calea către fișiere
    taxonomy_path = "insurance_taxonomy.csv"
    companies_path = "ml_insurance_challenge.csv"
    output_path = "annotated_companies1.csv"

    print("=" * 80)
    print("CLASIFICATOR DE TAXONOMIE PENTRU ASIGURĂRI")
    print("=" * 80)

    # Încarc datele
    print("\nÎncărcarea datelor...")
    start_time = time.time()
    taxonomy_df = pd.read_csv(taxonomy_path)
    companies_df = pd.read_csv(companies_path)
    data_load_time = time.time() - start_time

    print(f"Au fost încărcate {len(taxonomy_df)} etichete de taxonomie")
    print(f"Au fost încărcate {len(companies_df)} companii")
    print(f"Date încărcate în {data_load_time:.2f} secunde")

    # Inițializez clasificatorul
    print("\nInițializarea clasificatorului...")
    start_time = time.time()
    classifier = InsuranceTaxonomyClassifier(
        taxonomy_path=taxonomy_path,
        threshold=0.15
    )
    init_time = time.time() - start_time
    print(f"Clasificator inițializat în {init_time:.2f} secunde")

    # Evalueaz pe un eșantion înainte de clasificarea completă
    print("\nEvaluarea performanței clasificatorului pe un eșantion...")
    evaluation = classifier.evaluate_performance(companies_df, sample_size=200)

    # Clasific toate companiile
    print("\nClasificarea tuturor companiilor...")
    results_df = classifier.classify_companies(companies_df)

    # Salvez rezultatele
    print(f"\nSalvarea setului de date adnotat în {output_path}...")
    results_df.to_csv(output_path, index=False)

    print("\nProcesul de clasificare finalizat cu succes!")


if __name__ == "__main__":
    main()

CLASIFICATOR DE TAXONOMIE PENTRU ASIGURĂRI

Încărcarea datelor...
Au fost încărcate 220 etichete de taxonomie
Au fost încărcate 9494 companii
Date încărcate în 0.07 secunde

Inițializarea clasificatorului...
[16:51:22] Inițializare clasificator...
[16:51:22] Procesare 220 etichete din taxonomie...
[16:51:22] Clasificator inițializat cu succes!
Clasificator inițializat în 0.02 secunde

Evaluarea performanței clasificatorului pe un eșantion...
[16:51:22] Evaluarea performanței pe un eșantion de 200 companii...
[16:51:54] Evaluare finalizată în 31.28 secunde
[16:51:54] Rezultate evaluare:
  - Număr mediu de potriviri per companie: 45.91
  - Etichete unice atribuite: 217 (98.6% din taxonomie)
  - Top 5 etichete atribuite:
    - Fishing and Hunting Services: 44
    - Oil and Fat Manufacturing: 42
    - Training Services: 8
    - Sand and Gravel Mining: 8
    - Consulting Services: 7

Clasificarea tuturor companiilor...
[16:51:54] Începerea clasificării pentru 9494 companii...
[16:51:54] Pro